# Pipeline ETL - Sistema de Energia
## Extração, Transformação e Carga de Dados de Consumo

In [ ]:
import sys
import os
sys.path.append('../')

import pandas as pd
import numpy as np
import sqlite3
from src.utils import (
    gerar_dados_consumidores,
    gerar_dados_tarifas,
    gerar_dados_consumo,
    gerar_dados_picos
)

# Criar pastas necessárias
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

print(' Bibliotecas importadas com sucesso!')

## 1. EXTRAÇÃO (Extract)
### Gerando dados simulados

In [ ]:
# Gerar dados de consumidores
df_consumidores = gerar_dados_consumidores(n=100)
print(f' Consumidores gerados: {len(df_consumidores)}')
df_consumidores.head()

In [ ]:
# Gerar dados de tarifas
df_tarifas = gerar_dados_tarifas()
print(f' Tarifas geradas: {len(df_tarifas)}')
df_tarifas

In [ ]:
# Gerar dados de consumo (30 dias)
df_consumo = gerar_dados_consumo(df_consumidores, dias=30)
print(f' Registros de consumo gerados: {len(df_consumo):,}')
df_consumo.head()

In [ ]:
# Gerar dados de picos
df_picos = gerar_dados_picos(dias=30)
print(f' Registros de picos gerados: {len(df_picos)}')
df_picos.head()

## 2. TRANSFORMAÇÃO (Transform)
### Limpeza e enriquecimento dos dados

In [ ]:
# Converter data_hora para datetime
df_consumo['data_hora'] = pd.to_datetime(df_consumo['data_hora'])
df_picos['data_hora'] = pd.to_datetime(df_picos['data_hora'])

# Adicionar colunas derivadas
df_consumo['data'] = df_consumo['data_hora'].dt.date
df_consumo['hora'] = df_consumo['data_hora'].dt.hour
df_consumo['dia_semana'] = df_consumo['data_hora'].dt.day_name()
df_consumo['mes'] = df_consumo['data_hora'].dt.month

print(' Transformações aplicadas!')
df_consumo.head()

In [ ]:
# Verificar qualidade dos dados
print(' Verificação de Qualidade dos Dados\n')
print('Valores nulos:')
print(df_consumo.isnull().sum())
print('\nEstatísticas descritivas:')
df_consumo[['consumo_kwh', 'custo_total']].describe()

## 3. CARGA (Load)
### Salvando dados no banco SQLite

In [ ]:
# Conectar ao banco de dados
conn = sqlite3.connect('../data/energia.db')
print(' Conectado ao banco de dados')

# Carregar schema com encoding UTF-8
with open('../sql/schema.sql', 'r', encoding='utf-8') as f:
    schema = f.read()
    conn.executescript(schema)
    print(' Schema criado')

# Carregar dados nas tabelas
df_consumidores.to_sql('consumidores', conn, if_exists='replace', index=False)
print(f' {len(df_consumidores)} consumidores carregados')

df_tarifas.to_sql('tarifas', conn, if_exists='replace', index=False)
print(f' {len(df_tarifas)} tarifas carregadas')

df_consumo.to_sql('consumo', conn, if_exists='replace', index=False)
print(f' {len(df_consumo):,} registros de consumo carregados')

df_picos.to_sql('picos', conn, if_exists='replace', index=False)
print(f' {len(df_picos)} registros de picos carregados')

conn.close()
print('\n Pipeline ETL concluído com sucesso!')

## 4. Salvar dados processados em CSV

In [ ]:
# Salvar em CSV para análises futuras
df_consumo.to_csv('../data/processed/consumo_processado.csv', index=False)
df_consumidores.to_csv('../data/processed/consumidores.csv', index=False)
df_tarifas.to_csv('../data/processed/tarifas.csv', index=False)
df_picos.to_csv('../data/processed/picos.csv', index=False)

print(' Dados salvos em CSV!')